# LEE Analyzer notebook

This notebook takes in the output directory of the jobs and convert it into a more flat pandas dataframe. 
Data from the root files will be partially processed to fields that are convenient to plot.
The resulting dataframe will be pickled.

## Imports & Constants

In [1]:
import math
import glob
import time
import os
import sys
import numpy as np
import pandas as pd
from root_pandas import read_root
from collections import OrderedDict

from helpfunction import safely_reduce_dtype,reduce_mem_usage,sciNot,CheckBorderTPC,CheckBorderFixed

In [2]:
pd.options.display.max_columns = 999
gr      = 1.618
nan     = -2147483648
min_root_size = 20000 # Skip root files smaller than x bytes

mass_p= 0.93827 #GeV
mass_e= 0.00511 #GeV

# LAr EM showers
R_moliere =  9.5 # cm
X_o       = 13.9 # cm
E_c       = 0.035# GeV

In [3]:
# Which plane do we want to use:
plane = 2 # collection plane only

# Fiducial volume borders in x,y,z:
fid_arr= [[10,10],[20,20],[10,50]]
# Fiducial volume for the end points of tracks
fid_min = 10
# Percentage cut for shower containment
sh_cont_percent = .75

# Minimum reconstructable energies:
min_e = 0.02+mass_e # 20MeV
min_p = 0.04+mass_p # 40MeV

# Flat columns we want to copy from the original dataframe:
flat_columns = ['nu_pdg','nu_E','true_vx_sce','true_vy_sce','true_vz_sce',
                'distance','category','vx','vy','vz','bnbweight','passed']

vec_columns = ['shower_open_angle','shower_length','shower_start_x','shower_start_y','shower_start_z',
              'shower_dir_x','shower_dir_y','shower_dir_z','shower_pca',

              'track_start_x','track_start_y','track_start_z','track_end_x','track_end_y','track_end_z',
              'track_dir_x','track_dir_y','track_dir_z','track_pca',
              'predict_em','predict_mu','predict_cos','predict_pi','predict_p',

              'nu_daughters_pdg','nu_daughters_E','nu_daughters_px','nu_daughters_py','nu_daughters_pz',
              'nu_daughters_endx','nu_daughters_endy','nu_daughters_endz',
              'true_shower_pdg','true_shower_x_sce','true_shower_y_sce','true_shower_z_sce','true_shower_depE'
              ]

In [4]:
# Input directory
# Can be on gpvms, example: '/run/user/1000/gvfs/sftp:host=uboonegpvm02.fnal.gov,user=wvdp/uboone/data/users/wvdp/v06_26_01_10/data_bnb_a_1e0p/*/*.root'
# Local will be faster, avoid using pnfs scratch

inputlist = glob.glob('/home/wouter/Templates/nue/*/*.root')
filelist  = []
for fname in inputlist:
    if (os.stat(fname).st_size)<min_root_size:
            print('File',fname,'was corrupt. Size:',os.stat(fname).st_size/1000, 'kb, skipping to next file.')
    else:
        filelist.append(fname)
print(len(filelist),'valid ROOT files collected.')

File /home/wouter/Templates/nue/3396777_221/PandoraLEEAnalyzer.root was corrupt. Size: 0.434 kb, skipping to next file.
253 valid ROOT files collected.


## Classes

## Functions

### Help Functions

### Selection Functions

In [5]:
# Signal Definition 1e0p 
def true_thresholds_1e0p(row):
    if CheckBorderTPC(row['true_vx_sce'],row['true_vy_sce'],row['true_vz_sce'],fid_arr):
        passed_e=False
        for pdg,E in zip(row["nu_daughters_pdg"],row["nu_daughters_E"]):
            if pdg==11 and E>min_e:
                return True
    return False

### Feature functions

In [6]:
# Verifies if the event passed the flash precuts.
def FlashPrecut(row):
    flash_ok = False
    t_start = 3.2
    t_end   = t_start+1.6
    min_PE  = 50

    for time,PE in zip(row['flash_time'],row['flash_PE']):
        if time>t_start and time<t_end and PE>min_PE:
            flash_ok = True
    return pd.Series({"flash_precut": flash_ok}) 



# Reconstructed energy using collection plane
def CalcRecoE(row):
    reco_energy = 0
    total_nhits = 0 # sum over all the planes
    shower_nhits = np.zeros(row['n_showers']) 
    shower_energy = np.zeros(row['n_showers'])
    track_nhits = np.zeros(row['n_tracks'])
    track_energy = np.zeros(row['n_tracks'])
    
    for tr, tr_E, nhits in zip(range(row['n_tracks']),row['track_energy_hits'],row["track_nhits"]):
        reco_energy+=tr_E[plane]
        track_nhits[tr] = int(nhits[plane])
        track_energy[tr] = tr_E[plane]
        total_nhits+=int(sum(nhits))
        
    for sh, sh_E, nhits in zip(range(row['n_showers']),row['shower_energy'],row["shower_nhits"]):
        reco_energy+=sh_E[plane]
        shower_nhits[sh] = int(nhits[plane])
        shower_energy[sh] = sh_E[plane]
        total_nhits+=int(sum(nhits))
        
    return pd.Series({"reconstructed_energy": reco_energy, "total_nhits": total_nhits,
                      "shower_nhits": shower_nhits, "shower_energy": shower_energy,
                      "track_nhits": track_nhits, "track_energy": track_energy})



# Add info about the pandora particle hierarchy
def DaughterInfo(row):
    nu_shower_ids, nu_track_ids                = row['nu_shower_ids'],row['nu_track_ids']
    nu_shower_daughters, nu_track_daughters    = list(row['nu_shower_daughters']),list(row['nu_track_daughters'])

    showerdaughter = []
    for sh in nu_shower_daughters:
        if len(sh)==0:
            showerdaughter.append(0)
        elif len(sh)==1:
            if sh[0] in nu_shower_ids:
                showerdaughter.append(1)
            elif sh[0] in nu_track_ids:
                showerdaughter.append(2)
        else:
            showerdaughter.append(3)
            
    trackdaughter = []
    for tr in nu_track_daughters:
        if len(tr)==0:
            trackdaughter.append(0)
        elif len(tr)==1:
            if tr[0] in nu_shower_ids:
                trackdaughter.append(1)
            elif tr[0] in nu_track_ids:
                trackdaughter.append(2)
        else:
            trackdaughter.append(3)
    return pd.Series({"shower_daughter": showerdaughter, "track_daughter": trackdaughter})     



# Calculates the true end point for electron showers, for 95% of energy
def showerTrueEnd(row):
    if 11 not in row['nu_daughters_pdg']:
        return pd.Series({"true_shower_endx": -999.0, "true_shower_endy": -999.0, "true_shower_endz": -999.0,
                      "true_shower_tmax_x": -999.0, "true_shower_tmax_y": -999.0, "true_shower_tmax_z": -999.0})
    i_daughter = np.nonzero(row['nu_daughters_pdg']==11)
    if len(i_daughter[0])>1:
        print('More than 1 true electron daughter')
    i_daughter = i_daughter[0][0]
    
    E_ratio = (row['nu_daughters_E'][i_daughter])/E_c
    t_max = np.log(E_ratio)-1.0
    length = (t_max+0.08*18+9.6)*X_o
    #print('E_ratio',E_ratio,'E',row['nu_daughters_E'][i_daughter],' t_max',t_max,'length',length)
    direction = np.array([row['nu_daughters_px'][i_daughter],row['nu_daughters_py'][i_daughter],row['nu_daughters_pz'][i_daughter]])
    true_shower_start = np.array([row['nu_daughters_vx'][i_daughter],row['nu_daughters_vy'][i_daughter],row['nu_daughters_vz'][i_daughter]])
    true_shower_end = true_shower_start+length*direction/np.linalg.norm(direction)
    true_shower_tmax = true_shower_start+(t_max*X_o)*direction/np.linalg.norm(direction)
    
    return pd.Series({"true_shower_endx": true_shower_end[0], "true_shower_endy": true_shower_end[1], "true_shower_endz": true_shower_end[2],
                      "true_shower_tmax_x": true_shower_tmax[0], "true_shower_tmax_y": true_shower_tmax[1], "true_shower_tmax_z": true_shower_tmax[2]})



# Calculates the percentage of sps inside the fiducial volume and the percentage of deposited collection plane charge
def ContainedRatio(row):
    d=0.01
    n=0.0
    
    for x,y,z,q in zip(row['shower_sp_x'],row['shower_sp_y'],row['shower_sp_z'],row['shower_sp_int']):
        d+=q
        if CheckBorderTPC(x,y,z,fid_arr):
            n+=q

    return pd.Series({"shower_containment_q": n/d}) 



def CC_daughter_E(row):
    CC_daughter_i = np.nonzero(np.in1d(row['nu_daughters_pdg'], [11,-11,13,-13]))[0]
    CC_daughter_E = -1
    if len(CC_daughter_i)>0:
        if len(CC_daughter_i)>2:
            'Multiple electron/muon daughters found!'
        else:      
            CC_daughter_E = row['nu_daughters_E'][CC_daughter_i[0]]
    return pd.Series({"CC_daughter_E": CC_daughter_E})  



# Returns the ratio of collection charge of the first part and the second part of the summed shower.
def ShowerChargeProfile(row):
    x,y,z = row['vx'],row['vy'],row['vz']
    sps_x_arr,sps_y_arr,sps_z_arr,sps_int_arr = row['shower_sp_x'],row['shower_sp_y'],row['shower_sp_z'],row['shower_sp_int']
    
    center= np.array([0.0,0.0,0.0])
    total_Q = 0.0
    for sps_x,sps_y,sps_z,sps_int in zip(sps_x_arr,sps_y_arr,sps_z_arr,sps_int_arr):
        center+=np.array([sps_x,sps_y,sps_z])*sps_int
        total_Q+=sps_int
    center/=total_Q
    norm = (center-np.array([x,y,z])) / np.linalg.norm(center-np.array([x,y,z]))
    
    distance = []
    for sps_x,sps_y,sps_z in zip(sps_x_arr,sps_y_arr,sps_z_arr):
        distance.append( np.dot([sps_x-x,sps_y-y,sps_z-z],norm) )
        
    y,x = np.histogram( distance, weights = sps_int_arr )
    l = len(y)/2
    return pd.Series({"shower_sp_profile": np.mean(y[:l])/np.mean(y[-l:])})  


# Returns the dedx and the number of hits it had to compute this.
def CalcDedx(row):
    shower_dedx_hits = np.zeros(row['n_showers']) 
    shower_dedx      = np.zeros(row['n_showers'])
    shower_dedx_sum  = np.zeros(row['n_showers']) 
    
    track_dedx_hits  = np.zeros(row['n_tracks'])
    track_dedx       = np.zeros(row['n_tracks'])
    track_dedx_sum   = np.zeros(row['n_tracks'])
    
    
    for tr, tr_E, nhits in zip(range(row['n_tracks']),row['track_dEdx'],row["track_dEdx_hits"]):
        track_dedx_hits[tr] = len(nhits)
        track_dedx_sum[tr] += sum(nhits)
        track_dedx[tr]      = tr_E[plane]
        
    for sh, sh_E, nhits in zip(range(row['n_showers']),row['shower_dEdx'],row["shower_dEdx_hits"]):
        shower_dedx_hits[sh] = len(nhits)
        shower_dedx_sum[sh] += sum(nhits)
        shower_dedx[sh]      = sh_E[plane]
        
    shower_dedx_sum/=row['n_showers']
    track_dedx_sum/=row['n_tracks']
    
    return pd.Series({"shower_dedx_hits": shower_dedx_hits, "shower_dedx": shower_dedx, "shower_dedx_avg": shower_dedx_sum,
                      "track_dedx_hits": track_dedx_hits, "track_dedx": track_dedx, "track_dedx_avg": track_dedx_sum})

def OpticalInfo(row):
    flash_PE=0
    flash_time=0
    if np.any(row['flash_passed']!=-1):
        flash_PE = row['flash_PE'][np.max(row['flash_passed'])]
        flash_time = row['flash_time'][np.max(row['flash_passed'])]
    return pd.Series({"flash_time": flash_time, "flash_PE": flash_PE})

def MatchedCleanup(row):
    matched_tracks= row['matched_tracks']
    matched_tracks[row['matched_tracks']==-2147483648] = 0
    
    matched_tracks_energy= row['matched_tracks_energy']
    matched_tracks_energy[matched_tracks_energy<-1] = 0
    
    matched_showers= row['matched_showers']
    matched_showers[row['matched_showers']==-2147483648] = 0
    
    matched_showers_energy= row['matched_showers_energy']
    matched_showers_energy[matched_showers_energy<-1] = 0
    
    return pd.Series({"matched_tracks": matched_tracks, "matched_tracks_energy": matched_tracks_energy,
                      "matched_showers": matched_showers, "matched_showers_energy": matched_showers_energy})

### Selection & Features dictionaries

BNB $\nu$ + cosmics 


selection: We want to keep only the passed events but no further cuts on truth information

In [7]:
# Selection uses and ordered dict
Nu_select_true = OrderedDict([("Passed analyzer selection", lambda x: x['passed']==1)])
Nu_select_reco = OrderedDict([])

# Features uses a list of functions
Nu_feature_list =[DaughterInfo, ContainedRatio, ShowerChargeProfile, CalcRecoE, CalcDedx, OpticalInfo,
                 CC_daughter_E, showerTrueEnd, MatchedCleanup] # The last row used true info

BNB $\nu_e$ intrinsic + cosmics 


selection: We want to do an efficiency plot, therefore we will need to have the signal definition selection. Non passed events need also to be filtered on that requirement before they are saved. 


In [8]:
# Selection uses and ordered dict
Nue_select_true = OrderedDict([('Signal Definition 1e0p',true_thresholds_1e0p)])
Nue_select_reco = OrderedDict([("Passed analyzer selection", lambda x: x['passed']==1)])

# Features uses a list of functions
Nue_feature_list =[DaughterInfo, ContainedRatio, ShowerChargeProfile, CalcRecoE, CalcDedx, OpticalInfo,
                 CC_daughter_E, showerTrueEnd, MatchedCleanup] # The last row used true info

BNB ext or BNB data samples


selection: we only want to keep passed events 


We need to be carefull not adding features that depend on truth information.

In [9]:
# Selection uses and ordered dict
Data_select_true = OrderedDict([("Passed analyzer selection", lambda x: x['passed']==1)])
Data_select_reco = OrderedDict([])

# Features uses a list of functions
Data_feature_list =[DaughterInfo,FlashPrecut,ContainedRatio,ShowerChargeProfile,CalcRecoE,CalcDedx,OpticalInfo]

### Main Function: loadData

In [ ]:
def loadData(filelist,                  # List of input files
             selection_true,            # Function dict that contains the true based selection, applied before saving the compact complete dataframw
             selection_reco,            # Ordered dictionary with bool function that act on rows
             featurelist,               # list of functions returning columns
             maxf=2,                    # Maximum number of files to loop over
             outputname='output'        # Name of the final picle file
            ):
    
    chunks = []                                 # list of small dataframes with all info
    chunks_nonpassed = []                       # list of small dataframes for failed event bookkeeping
    entries = 0                                 # entries before selection
    counter = np.zeros(len(selection_reco)+len(selection_true))      # counts number of events passing each stage
    total_pot = 0                               # total POT of the sample
    
    nfiles=len(filelist)
    if maxf<nfiles:
        nfiles=maxf
        
    print ('Start to load entries from',nfiles,'files.\n')
    start_time = time.time()
    
    progress=0
    for i_f,fname in enumerate(filelist[:nfiles]):
        if(i_f % math.ceil(nfiles/10)==0 ):
            print ('Progress:',progress,'%.')
            progress+=10
            
        # Store the POT of the sample
        total_pot+=read_root(fname,"robertoana/pot",columns='pot').sum()
        
        dftemp=read_root(fname,"robertoana/pandoratree")
        entries+=len(dftemp.index)
        
        # Truth based selection
        for index,(key, value) in enumerate(selection_true.items()):
            dftemp = dftemp[dftemp.apply(value,axis=1)]
            counter[index]+=len(dftemp.index)
            
        # Store some basic things about events that did not pass the selection! (but passed the truth selection)
        dftemp_nonpassed = dftemp[dftemp['passed']==0][['nu_pdg','nu_E','true_vx_sce','true_vy_sce','true_vz_sce',
                                                        'category','bnbweight']]
        # This compact dataframe should also have some optical information.
        dftemp_nonpassed = pd.concat([dftemp_nonpassed, dftemp[dftemp['passed']==0].apply(OpticalInfo,axis=1)], axis=1)
        
        # Reco based selection
        for index,(key, value) in enumerate(selection_reco.items()):
            dftemp = dftemp[dftemp.apply(value,axis=1)]
            counter[index+len(selection_reco)]+=len(dftemp.index)
        
        # introduce the new flattened dataframe:
        df_new = dftemp[flat_columns]
        #df_new['shower_dEdx_hits'] = dftemp['shower_dEdx_hits']
        #df_new['shower_dEdx'] = dftemp['shower_dEdx']
        
        # Reduce the dataframe size of the vector columns
        for col in vec_columns:
            df_new[col]=dftemp[col].apply(safely_reduce_dtype)
        
        # add new features to it
        for value in featurelist:
            df_new = pd.concat([df_new, dftemp.apply(value,axis=1)], axis=1)
            
        chunks.append(df_new)
        chunks_nonpassed.append(dftemp_nonpassed)
       
    end_time = time.time()
    
    print('\nSummary:')
    print (entries,'entries were loaded from',nfiles,'files, corresponding to',str(total_pot[0]),'POT.')
    for key,counts in zip(list(selection_true.keys())+list(selection_reco.keys()),counter):
        print(counts,'Passed "',key,'" stage of selection.')
        
    print('\nLoading took ',sciNot(end_time-start_time),' seconds.')       
    print('Concatenating output dataframes')
    df = pd.concat(chunks,ignore_index=True,copy=False) 
    df_nonpassed = pd.concat(chunks_nonpassed,ignore_index=True,copy=False) 
    print ('Final dataframe has',len(df.index),'entries.')
    
    #Reduce the dataframe size or the non vector columns
    df,_ = reduce_mem_usage(df)
    df_nonpassed,_ = reduce_mem_usage(df_nonpassed)
    
    df.to_pickle('Input/'+outputname+'.pckl')
    df_nonpassed.to_pickle('Input/'+outputname+'_nonpassed.pckl')
    end2_time = time.time()
    print('Pickling took ',sciNot(end2_time-end_time),' seconds.')

## Load dataframe and save to Pickle

In [ ]:
loadData(filelist,          # List of input files
             Nue_select_true,           # Function dict that contains the true based selection, applied before saving the compact complete dataframw
             Nue_select_reco,           # Ordered dictionary with bool function that act on rows
             Nue_feature_list,          # list of functions returning columns
             maxf=300,                  # Maximum number of files to loop over
             outputname='nue'           # Name of the final picle file
            )
print('Done!')

Start to load entries from 253 files.

Progress: 0 %.


/home/wouter/anaconda3/envs/rootenv/lib/python3.4/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/wouter/anaconda3/envs/rootenv/lib/python3.4/site-packages/ipykernel/__main__.py:143: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/wouter/anaconda3/envs/rootenv/lib/python3.4/site-packages/ipykernel/__main__.py:134: RuntimeWarning: invalid value encountered in true_divide
/home/wouter/anaconda3/envs/rootenv/lib/python3.4/site-packages/ipykernel/__main__.py:143: RuntimeWarning: invalid value encountered in float_scalars
/home/wouter/anaconda3/envs/rootenv/lib/python3.4/site-packages/ipykernel/__main__.py:143: RuntimeWarning: divide by zero encountered in

More than 1 true electron daughter


### Done!